In [2]:
! uv pip install langchain langchain-openai openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.6 environment at: C:\Users\junjo\Desktop\PRO\LLMops\.venv
Resolved 76 packages in 1.61s
Prepared 1 package in 112ms
Installed 1 package in 27ms
 + langchain-openai==1.1.7


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")

### Data Ingestion

In [3]:
from langchain_community.document_loaders import TextLoader

In [4]:
loader=TextLoader(r'\Users\junjo\Desktop\PRO\LLMops\data\agentic_ai.txt', encoding='utf-8')
documents=loader.load()
documents[0].page_content[:500]

'Agentic AI refers to autonomous systems that plan, reason, and act independently to achieve goals.\nUnlike traditional AI, these agents break down complex tasks into manageable steps.\nThey execute actions using tools like APIs, databases, or web services.\nAgentic systems observe their environment to gather real-time information.\nThey make decisions based on reasoning powered by large language models.\nAgents learn from outcomes through reflection and feedback loops.\nShort-term memory handles curre'

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
)

In [11]:
text_chunks= text_splitter.split_documents(documents)
print(len(text_chunks))
text_chunks

13


[Document(metadata={'source': '\\Users\\junjo\\Desktop\\PRO\\LLMops\\data\\agentic_ai.txt'}, page_content='Agentic AI refers to autonomous systems that plan, reason, and act independently to achieve goals.\nUnlike traditional AI, these agents break down complex tasks into manageable steps.'),
 Document(metadata={'source': '\\Users\\junjo\\Desktop\\PRO\\LLMops\\data\\agentic_ai.txt'}, page_content='They execute actions using tools like APIs, databases, or web services.\nAgentic systems observe their environment to gather real-time information.'),
 Document(metadata={'source': '\\Users\\junjo\\Desktop\\PRO\\LLMops\\data\\agentic_ai.txt'}, page_content='They make decisions based on reasoning powered by large language models.\nAgents learn from outcomes through reflection and feedback loops.'),
 Document(metadata={'source': '\\Users\\junjo\\Desktop\\PRO\\LLMops\\data\\agentic_ai.txt'}, page_content='Short-term memory handles current tasks while long-term memory stores patterns.\nThis memor

In [7]:
! uv pip install faiss-cpu

Using Python 3.12.6 environment at: C:\Users\junjo\Desktop\PRO\LLMops\.venv
Audited 1 package in 6ms


In [8]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [9]:
embeddings= OpenAIEmbeddings()

C:\Users\junjo\AppData\Local\Temp\ipykernel_15596\2908611556.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  embeddings= OpenAIEmbeddings()


In [12]:
vectorstore=FAISS.from_documents(text_chunks,embeddings)

In [13]:
retriever=vectorstore.as_retriever()

In [14]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=5)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)


Document 1:
Agentic AI refers to autonomous systems that plan, reason, and act independently to achieve goals.
Unlike traditional AI, these agents break down complex tasks into manageable steps.
--------------------------------------------------
Document 2:
Ethical alignment ensures actions match human values.
Agentic AI augments workers rather than fully replacing them.
Early adopters report 30-50% efficiency gains.
--------------------------------------------------
Document 3:
Open-source tools speed up developer experimentation.
Security limits actions via permissioned tool access.
Agentic AI evolves to tackle real-world complexity intelligently.
--------------------------------------------------
Document 4:
Adaptability allows agents to respond to dynamic changes or new data.
Multi-agent systems feature specialized agents that collaborate on complex goals.
--------------------------------------------------
Document 5:
Challenges include ensuring safety and handling unpredictable ed

In [15]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [16]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [17]:
from langchain_core.output_parsers import StrOutputParser

In [18]:
output_parser= StrOutputParser()

In [19]:
from langchain_openai import ChatOpenAI
llm_model=ChatOpenAI(model="gpt-4o-mini")

In [21]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm_model
    | output_parser
)


In [23]:
rag_chain.invoke("What is the Key Characteristics of Agentic AI?")

'Key characteristics of Agentic AI include the ability to operate autonomously by planning, reasoning, and acting independently to achieve specific goals. Unlike traditional AI, Agentic AI can break down complex tasks into manageable steps, allowing for more efficient execution. These systems are designed with ethical alignment, ensuring that their actions align with human values. Agentic AI augments human workers instead of fully replacing them, leading to significant efficiency gains, reportedly between 30-50% for early adopters. The technology is adaptable, allowing agents to respond to dynamic changes and new data effectively. Additionally, multi-agent systems can incorporate specialized agents that collaborate on complex objectives. Open-source tools facilitate faster experimentation by developers, while security measures can limit actions through permissioned access. Overall, Agentic AI evolves to address real-world complexities intelligently.'